In [1]:
import pickle
with open('stanford_biology_all.pkl', 'rb') as f:
    stanford_biology_all = pickle.load(f)

In [2]:
stanford_biology_all[0]

[('The', 'DT'),
 ('construct', 'NN'),
 ('retained', 'VBD'),
 ('the', 'DT'),
 ('native', 'JJ'),
 ('signal', 'NN'),
 ('sequence', 'NN'),
 ('to', 'TO'),
 ('allow', 'VB'),
 ('secretion', 'NN'),
 ('of', 'IN'),
 ('recombinant', 'JJ'),
 ('protein', 'NN'),
 ('into', 'IN'),
 ('the', 'DT'),
 ('culture', 'NN'),
 ('medium', 'NN'),
 ('.', '.')]

In [15]:
import dataset
bio = dataset.get_biology(limit_train=0, limit_val=0, limit_test=None, check_pos=False)

In [16]:
test_tags = bio['test_tags']
test_tags[0]

[['The', 'DT'],
 ['construct', 'NN'],
 ['retained', 'VBD'],
 ['the', 'DT'],
 ['native', 'JJ'],
 ['signal', 'NN'],
 ['sequence', 'NN'],
 ['to', 'TO'],
 ['allow', 'VB'],
 ['secretion', 'NN'],
 ['of', 'IN'],
 ['recombinant', 'JJ'],
 ['protein', 'NN'],
 ['into', 'IN'],
 ['the', 'DT'],
 ['culture', 'NN'],
 ['medium', 'NN'],
 ['.', '.']]

In [70]:
correct = 0
incorrect = 0
correct_sents = 0
incorrect_sents = 0
all_bio_pos = []
all_truth_pos = []
for i in range(len(stanford_biology_all)):
    bio_sent = stanford_biology_all[i]
    truth_sent = test_tags[i]
    # bio_sent_just_pos = [tup[1] for tup in bio_sent]
    # all_bio_pos += bio_sent_just_pos
    # truth_sent_just_pos = [tup[1] for tup in truth_sent]
    # all_truth_pos += truth_sent_just_pos
    # print(bio_sent)
    # print(truth_sent)
    # print(bio_sent_just_pos)
    # print(truth_sent_just_pos)
    s1 = truth_sent
    s2 = bio_sent
    fixed_s2 = []
    ind_s1 = 0
    ind_s2 = 0
    while ind_s1 < len(s1) or ind_s2 < len(s2):
        # print(ind_s1, ind_s2, s1[ind_s1], s2[ind_s2])
        for delta in range(1, 5):
            s1_joined = s1[ind_s1][0].replace('(', '-LRB-').replace(')', '-RRB-')
            s2_joined = ' '.join([x[0] for x in s2[ind_s2:ind_s2+delta]])

            if s1_joined == s2_joined:
                fixed_s2.append((s2_joined, [s2[ind_s2+x][1] for x in range(delta)]))
                ind_s1 += 1
                ind_s2 += delta
                break
        else:
            print(i)
            print(ind_s1, ind_s2, s1[ind_s1], s2[ind_s2])
            print(s1)
            print(s2)
            assert False
    bio_sent = fixed_s2
    # if len(bio_sent_just_pos) != len(truth_sent_just_pos):
    #     print(bio_sent)
    #     print(truth_sent)
    #     print()
    #     continue

    if all(any(x == truth_sent[j][1] for x in bio_sent[j][1]) for j in range(len(bio_sent))):
        correct_sents += 1
    else:
        incorrect_sents += 1

    for j in range(len(bio_sent)):
        if len(bio_sent[j][1]) == 1:
            if bio_sent[j][1][0] == truth_sent[j][1]:
                correct += 1
            else:
                incorrect += 1
        else:
            if any(x == truth_sent[j][1] for x in bio_sent[j][1]):
                correct += 1
            else:
                incorrect += 1
    
    # print()

# all_bio_pos = sum(len(x) for x in test_tags)
# all_bio_pos = len(test_tags)
all_bio_pos = sum([len(sent) for sent in test_tags])

# print(f'correct={correct}, incorrect={incorrect}, all_bio_pos={all_bio_pos}')
assert(correct + incorrect == all_bio_pos)
assert(correct_sents + incorrect_sents == len(stanford_biology_all))
print(f'correct: {correct}/{all_bio_pos} ({100*correct/all_bio_pos}%), incorrect: {incorrect}/{all_bio_pos} ({100*incorrect/all_bio_pos}%)')
print(f'correct_sents: {correct_sents}/{len(stanford_biology_all)} ({100*correct_sents/len(stanford_biology_all)}%), incorrect_sents: {incorrect_sents}/{len(stanford_biology_all)} ({100*incorrect_sents/len(stanford_biology_all)}%)')

correct: 674238/818428 (82.3820788145078%), incorrect: 144190/818428 (17.617921185492186%)
correct_sents: 2549/31810 (8.013203395158754%), incorrect_sents: 29261/31810 (91.98679660484125%)
